In [3]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="phpipamadmin",
  database="assetdb"
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)
mycursor.close()

True

In [5]:
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE assetdb_main (srcaddr INT UNSIGNED, srcport MEDIUMINT UNSIGNED, first BIGINT UNSIGNED, last BIGINT UNSIGNED, protocol TINYINT UNSIGNED, flows SMALLINT UNSIGNED, packets MEDIUMINT UNSIGNED, bytes INT UNSIGNED, PRIMARY KEY(`srcaddr`)) \
ENGINE=InnoDB DEFAULT CHARSET=utf8 PARTITION BY HASH(srcaddr) partitions 64;")

mycursor.close()

True

In [4]:
import socket  
import struct  
  

ip = '127.0.0.1'  
int_ip = struct.unpack('!I', socket.inet_aton(ip))[0]  
print(int_ip)  
str_ip = socket.inet_ntoa(struct.pack('!I', int_ip))  
print(str_ip) 


2130706433
127.0.0.1


In [5]:
def protocol_to_number(protocol):
    if protocol == 'TCP':
        num = 1
    elif protocol=='UDP':
        num = 2
    elif protocol == 'ssh':
        num = 3
    return num

In [6]:
# Python read all files from a directory in order
import os

f = open("./data/netflow_name.txt", 'w')
path = "./data/netflow/"
files = os.listdir(path)
files.sort()

s = []

for file_name in files:
    if not os.path.isdir(path + file_name):
        f_name = str(file_name)
        print(f_name)
        s.append(f_name)
        f.write(f_name + '\n')
for i in s:
    print(i)
print(s)

20200528140758.json
20200528141325.json
20200528141851.json
20200528142416.json
20200528142940.json
20200528143504.json
20200528144029.json
20200528144554.json
20200528145119.json
20200528145643.json
20200528150208.json
20200528150732.json
20200528151257.json
20200528151821.json
20200528152346.json
20200528152910.json
20200528153435.json
20200528153959.json
20200528154524.json
20200528155048.json
20200528155613.json
20200528160137.json
20200528160702.json
20200528161228.json
20200528161752.json
20200528162317.json
20200528162841.json
20200528163405.json
20200528163930.json
20200528164454.json
20200528165019.json
20200528165543.json
20200528170107.json
20200528170632.json
20200528171156.json
20200528171721.json
20200528172246.json
20200528172810.json
20200528173335.json
20200528173859.json
20200528174424.json
20200528174948.json
20200528175513.json
20200528180037.json
20200528180602.json
20200528181126.json
20200528181650.json
20200528182215.json
20200528182740.json
20200528183304.json


In [7]:
!pip install influxdb

In [8]:
from influxdb import InfluxDBClient
client = InfluxDBClient('localhost', 8086, 'root', '0318', 'assetdb_ts') 

In [14]:
print(client.get_list_database()) # 显示所有数据库名称
client.create_database('assetdb_ts') # 创建数据库
print(client.get_list_database()) # 显示所有数据库名称
client.drop_database('assetdb_ts') # 删除数据库
print(client.get_list_database()) # 显示所有数据库名称

[{'name': '_internal'}, {'name': 'assetdb_ts'}]
[{'name': '_internal'}, {'name': 'assetdb_ts'}]
[{'name': '_internal'}]


In [15]:
client.create_database('assetdb_ts') # 创建数据库
print(client.get_list_database()) # 显示所有数据库名称

[{'name': '_internal'}, {'name': 'assetdb_ts'}]


In [16]:
client = InfluxDBClient('localhost', 8086, 'root', '0318', 'assetdb_ts') 

In [17]:
json_body = [
    {
        "measurement": "time_series",
        "tags": {
            "ip": "10.10.10.10"
        },
        #"time": "2017-03-12T22:00:00Z",
        "fields": {
            "port": 80,
        }
    }
]

client.write_points(json_body)

True

In [ ]:
with open('data/test data/' + 'test_data.json') as record:
        new_record = json.load(record)

In [18]:
result = client.query('select * from time_series;')    
print("Result: {0}".format(result))

Result: ResultSet({'('time_series', None)': [{'time': '2020-06-28T17:34:23.157384Z', 'ip': '10.10.10.10', 'port': 80}]})


In [135]:
"""Tutorial for using pandas and the InfluxDB client."""

import argparse
import pandas as pd
import json

from influxdb import DataFrameClient


def main():
    """Instantiate the connection to the InfluxDB client."""
    user = 'root'
    password = '0318'
    dbname = 'assetdb_ts'
    protocol = 'line'

    client = DataFrameClient('localhost', 8086, user, password, dbname)

    print("Create pandas DataFrame")
    df = pd.DataFrame(data=list(range(30)),
                      index=pd.date_range(start='2014-11-16',
                                          periods=30, freq='H'), columns=['0'])
    print(df)
    with open('data/test data/' + 'test_data.json') as record:
        new_record = json.load(record)

        new_record = pd.DataFrame(data=new_record,                       
                                  columns=['srcaddr','srcport','First'])
        new_record['First'] = pd.to_datetime(new_record['First'], format='%Y%m%d%H%M%S')
        new_record.set_index('First', inplace=True)

        print("Create database: " + dbname)
        client.create_database(dbname)

        print("Write DataFrame")
        #tags = { "srcaddr": new_record[["srcaddr"]],  "srcport": new_record[["srcport"]]}
        #client.write_points(new_record, 'demo', tags = tags, protocol="json")
        client.write_points(new_record, 'demo', tag_columns=['srcaddr'], protocol=protocol)

        print("Read DataFrame")
        result = client.query("select * from demo")
        print("Result: {0}".format(result))

        print("Delete database: " + dbname)
        client.drop_database(dbname)


def parse_args():
    """Parse the args from main."""
    parser = argparse.ArgumentParser(
        description='example code to play with InfluxDB')
    parser.add_argument('--host', type=str, required=False,
                        default='localhost',
                        help='hostname of InfluxDB http API')
    parser.add_argument('--port', type=int, required=False, default=8086,
                        help='port of InfluxDB http API')
    return parser.parse_args()


if __name__ == '__main__':

    main()

Create pandas DataFrame
                      0
2014-11-16 00:00:00   0
2014-11-16 01:00:00   1
2014-11-16 02:00:00   2
2014-11-16 03:00:00   3
2014-11-16 04:00:00   4
2014-11-16 05:00:00   5
2014-11-16 06:00:00   6
2014-11-16 07:00:00   7
2014-11-16 08:00:00   8
2014-11-16 09:00:00   9
2014-11-16 10:00:00  10
2014-11-16 11:00:00  11
2014-11-16 12:00:00  12
2014-11-16 13:00:00  13
2014-11-16 14:00:00  14
2014-11-16 15:00:00  15
2014-11-16 16:00:00  16
2014-11-16 17:00:00  17
2014-11-16 18:00:00  18
2014-11-16 19:00:00  19
2014-11-16 20:00:00  20
2014-11-16 21:00:00  21
2014-11-16 22:00:00  22
2014-11-16 23:00:00  23
2014-11-17 00:00:00  24
2014-11-17 01:00:00  25
2014-11-17 02:00:00  26
2014-11-17 03:00:00  27
2014-11-17 04:00:00  28
2014-11-17 05:00:00  29
Create database: assetdb_ts
Write DataFrame
Read DataFrame
Result: defaultdict(<class 'list'>, {'demo':                                    srcaddr  srcport
2020-05-28 14:07:31+00:00   125.202.46.124    10110
2020-05-28 14:07:31+00:0

In [106]:
"""Tutorial for using pandas and the InfluxDB client."""

import argparse
import pandas as pd
import json

from influxdb import DataFrameClient


def main():
    """Instantiate the connection to the InfluxDB client."""
    user = 'root'
    password = '0318'
    dbname = 'assetdb_ts'
    protocol = 'line'

    client = DataFrameClient('localhost', 8086, user, password, dbname)
    with open('data/test data/' + 'test_data.json') as record:
        new_record = json.load(record)
        print(new_record)

        print("Create pandas DataFrame")
        df = pd.DataFrame(new_record,
                          columns=['srcaddr','srcport','First'])
            df['First'] = pd.to_datetime(df['First'], format='%Y%m%d%H%M%S')
        
        df = pd.DataFrame(new_record,
                          index=pd.to_datetime(new_record['First'], format='%Y%m%d%H%M%S'), 
                          columns=['srcaddr','srcport','First'])

        print(df)
        # df.set_index('First', inplace=True)
        
        
        print(df)
        print("Create database: " + dbname)
        client.create_database(dbname)

        print("Write DataFrame")
        client.write_points(df, 'demo', protocol=protocol)

        print("Write DataFrame with Tags")
        client.write_points(df, 'demo',
                            {'k1': 'v1', 'k2': 'v2'}, protocol=protocol)

        print("Read DataFrame")
        result = client.query("select * from demo")
        print("Result: {0}".format(result))

        print("Delete database: " + dbname)
        client.drop_database(dbname)


def parse_args():
    """Parse the args from main."""
    parser = argparse.ArgumentParser(
        description='example code to play with InfluxDB')
    parser.add_argument('--host', type=str, required=False,
                        default='localhost',
                        help='hostname of InfluxDB http API')
    parser.add_argument('--port', type=int, required=False, default=8086,
                        help='port of InfluxDB http API')
    return parser.parse_args()


if __name__ == '__main__':

    main()

[{'srcaddr': '57.216.112.001', 'srcport': 63975, 'First': 20200528140731, 'Last': 20200528140931, 'protocol': 'UDP', 'flows': 18, 'packets': 295, 'bytes': 482341}, {'srcaddr': '184.105.195.183', 'srcport': 38885, 'First': 20200528140731, 'Last': 20200528142533, 'protocol': 'TCP', 'flows': 3, 'packets': 112, 'bytes': 215847}, {'srcaddr': '125.202.46.124', 'srcport': 10110, 'First': 20200528140731, 'Last': 20200528142534, 'protocol': 'UDP', 'flows': 1, 'packets': 243, 'bytes': 115125}]
Create pandas DataFrame


ValueError: Shape of passed values is (3, 2), indices imply (1, 2)

In [102]:
import time
tupTime = time.localtime(20200528140731)
time.strftime("%Y%m%d%H%M%S", tupTime)

'6420990403015211'

In [89]:
"""Tutorial for using pandas and the InfluxDB client."""

import argparse
import pandas as pd
import json

from influxdb import DataFrameClient


def main():
    """Instantiate the connection to the InfluxDB client."""
    user = 'root'
    password = '0318'
    dbname = 'assetdb_ts'
    protocol = 'line'

    client = DataFrameClient('localhost', 8086, user, password, dbname)


    print("Create pandas DataFrame")
    df = pd.DataFrame(data=list(range(30)),
                      index=pd.date_range(start='2014-11-16',
                                          periods=30, freq='H'), columns=['0'])
    print(df)
    print("Create database: " + dbname)
    client.create_database(dbname)

    print("Write DataFrame")
    client.write_points(df, 'demo', protocol=protocol)

    print("Write DataFrame with Tags")
    client.write_points(df, 'demo',
                        {'k1': 'v1', 'k2': 'v2'}, protocol=protocol)

    print("Read DataFrame")
    result = client.query("select * from demo")
    print("Result: {0}".format(result))

    print("Delete database: " + dbname)
    client.drop_database(dbname)


def parse_args():
    """Parse the args from main."""
    parser = argparse.ArgumentParser(
        description='example code to play with InfluxDB')
    parser.add_argument('--host', type=str, required=False,
                        default='localhost',
                        help='hostname of InfluxDB http API')
    parser.add_argument('--port', type=int, required=False, default=8086,
                        help='port of InfluxDB http API')
    return parser.parse_args()


if __name__ == '__main__':

    main()

Create pandas DataFrame
                      0
2014-11-16 00:00:00   0
2014-11-16 01:00:00   1
2014-11-16 02:00:00   2
2014-11-16 03:00:00   3
2014-11-16 04:00:00   4
2014-11-16 05:00:00   5
2014-11-16 06:00:00   6
2014-11-16 07:00:00   7
2014-11-16 08:00:00   8
2014-11-16 09:00:00   9
2014-11-16 10:00:00  10
2014-11-16 11:00:00  11
2014-11-16 12:00:00  12
2014-11-16 13:00:00  13
2014-11-16 14:00:00  14
2014-11-16 15:00:00  15
2014-11-16 16:00:00  16
2014-11-16 17:00:00  17
2014-11-16 18:00:00  18
2014-11-16 19:00:00  19
2014-11-16 20:00:00  20
2014-11-16 21:00:00  21
2014-11-16 22:00:00  22
2014-11-16 23:00:00  23
2014-11-17 00:00:00  24
2014-11-17 01:00:00  25
2014-11-17 02:00:00  26
2014-11-17 03:00:00  27
2014-11-17 04:00:00  28
2014-11-17 05:00:00  29
Create database: assetdb_ts
Write DataFrame
Write DataFrame with Tags
Read DataFrame
Result: defaultdict(<class 'list'>, {'demo':                             0    k1    k2
2014-11-16 00:00:00+00:00   0  None  None
2014-11-16 00:00:0

In [ ]:
import time
import json

# update 1 X 1000000 records at a time
# Use List to collect the time of each insert operation for every 1 million records
# Insert thread pool configuration

time_records = []
time_records_opt = []

for json_name in s:
#    mydb.reconnect()
    mycursor = mydb.cursor()
    sql_update = "INSERT INTO p2_netflow (srcaddr, srcport, first, last, protocol, flows, packets, bytes) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE `last` = VALUES(`last`), `flows`= VALUES(`flows`) + `flows`, `packets`= VALUES(`packets`) + `packets`, `bytes`= VALUES(`bytes`) + `bytes`"

    start_time = time.time()
    flows = []
    with open('data/netflow/' + json_name) as record:
        new_record = json.load(record)
        new_record.sort(key=lambda k: (k.get('srcaddr', 0)))
        for flow in new_record:
            srcaddr = struct.unpack('!I', socket.inet_aton(flow["srcaddr"]))[0]
            protocol = protocol_to_number(flow["protocol"])
            flow_one = (srcaddr, flow["srcport"], flow["First"], flow["Last"], protocol, flow["flows"], flow["packets"], flow["bytes"])
            flows.append(flow_one)
        mycursor.executemany(sql_update, flows)
        print(mycursor.rowcount, "records inserted successful。")
        mycursor.close()
        mydb.commit()    # 数据表内容有更新，必须使用到该语句
#       mydb.close()
        end_time = time.time()
        consume_time = end_time - start_time
        time_records.append(consume_time)
        print(consume_time) 
        consume_time = '{:0>2s}'.format(str(int(consume_time // 3600))) \
               + ':{:0>2s}'.format(str(int((consume_time // 60) % 60))) \
               + ':{:0>2s}'.format(str(int(consume_time % 60)))
        time_records_opt.append(consume_time)
        print(consume_time) 
        
mydb.close()